In [63]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import nltk
import spacy
import en_core_web_sm
nlp = spacy.load('en_core_web_sm', disable = ['ner', 'parser'])
from gensim.parsing.preprocessing import remove_stopwords
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [64]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
data = pd.read_csv('/content/drive/Shareddrives/NLP/Team Blue NLP/train_cleaned.csv')
data.head()

,Unnamed: 0,id,comment_text,avg_count_caps,comment_cleaned,comment_tokenized,comment_cleaned_spell,comment_cleaned_no_stopwords,comment_cleaned_spell_no_stopwords,comment_cleaned_no_stopwords_lemm,comment_cleaned_spell_no_stopwords_lemm,comment_cleaned_lemm,comment_cleaned_spell_lemm,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,131272,be631fee6a996d52,List of Moroccan Dutch people \n\nPeople shoul...,0.054422,list of moroccan dutch people people should ...,"['List', 'of', 'Moroccan', 'Dutch', 'people', ...",list of moroccan dutch people people should ha...,list moroccan dutch people people wikipedia pa...,list moroccan dutch people people wikipedia pa...,list moroccan dutch people people wikipedia pa...,list moroccan dutch people people wikipedia pa...,list of moroccan dutch people people should ha...,list of moroccan dutch people people should ha...,0,0,0,0,0,0
1,55898,955b143f75755ac3,"Not much better, be careful should someone rep...",0.012658,not much better be careful should someone rep...,"['Not', 'much', 'better', 'be', 'careful', 'sh...",not much better be careful should someone repo...,much better careful someone report ban vandalism,much better careful someone report ban vandalism,much well careful someone report ban vandalism,much well careful someone report ban vandalism,not much good be careful should someone report...,not much good be careful should someone report...,0,0,0,0,0,0
2,86025,e623261850dd2aaa,I blocked the pair of you because you are not ...,0.022857,i blocked the pair of you because you are not ...,"['I', 'blocked', 'the', 'pair', 'of', 'you', '...",i blocked the pair of you because you are not ...,blocked pair write encyclopaedia plenty places...,blocked pair write encyclopaedia plenty places...,block pair write encyclopaedia plenty place pl...,block pair write encyclopaedia plenty place pl...,i block the pair of you because you be not her...,i block the pair of you because you be not her...,0,0,0,0,0,0
3,96930,068e0a8032a4cf5c,Thank You \n\nThanks for the link on the refer...,0.020833,thank you thanks for the link on the referen...,"['Thank', 'You', 'Thanks', 'for', 'the', 'link...",thank you thanks for the link on the reference...,thank thanks link reference desks crux orthodo...,thank thanks link reference desks crux orthodo...,thank thanks link reference desk crux orthodox...,thank thanks link reference desk crux orthodox...,thank you thank for the link on the reference ...,thank you thank for the link on the reference ...,0,0,0,0,0,0
4,82799,dd852a6da913a096,"I know of about 50,000 Bible Scholars who disa...",0.034146,i know of about _number_ bible scholars who di...,"['I', 'know', 'of', 'about', '_NUMBER_', 'Bibl...",i know of about _number_ bible scholars who di...,know _number_ bible scholars disagree yet libe...,know _number_ bible scholars disagree yet libe...,know _number_ bible scholar disagree yet liber...,know _number_ bible scholar disagree yet liber...,i know of about _number_ bible scholar who dis...,i know of about _number_ bible scholar who dis...,0,0,0,0,0,0


In [66]:
data.dropna(inplace=True)

# Truncated Dataset

In [67]:
positive = data[data['toxic']==1].head(10000)
negative = data[data['toxic']==0].head(10000)
data1 = pd.concat([positive, negative])
data1.shape

(20000, 19)

In [68]:
data1.head()

,Unnamed: 0,id,comment_text,avg_count_caps,comment_cleaned,comment_tokenized,comment_cleaned_spell,comment_cleaned_no_stopwords,comment_cleaned_spell_no_stopwords,comment_cleaned_no_stopwords_lemm,comment_cleaned_spell_no_stopwords_lemm,comment_cleaned_lemm,comment_cleaned_spell_lemm,toxic,severe_toxic,obscene,threat,insult,identity_hate
18,10537,1bccfd25bf571cde,"""\n\n Contested deletion \n\nThis page should ...",0.056881,contested deletion this page should not ...,"['Contested', 'deletion', 'This', 'page', 'sho...",contested deletion this page should not be spe...,contested deletion page speedily deleted speak...,contested deletion page speedily deleted speak...,contest deletion page speedily delete speaks t...,contest deletion page speedily delete speaks t...,contest deletion this page should not be speed...,contest deletion this page should not be speed...,1,0,1,0,1,1
26,49132,835ae075e5d44a3b,"Ricky81682, is a paid mole ScumBag \n\nThere ...",0.062992,ricky_datetime_2 is a paid mole scumbag th...,"['Ricky_DATETIME_2', 'is', 'a', 'paid', 'mole'...",ricky_datetime_2 is a paid mole scumbag there ...,ricky_datetime_2 paid mole scumbag question wh...,ricky_datetime_2 paid mole scumbag question wh...,ricky_datetime_2 pay mole scumbag question wha...,ricky_datetime_2 pay mole scumbag question wha...,ricky_datetime_2 be a paid mole scumbag there ...,ricky_datetime_2 be a paid mole scumbag there ...,1,0,1,0,1,0
50,77962,d0bb94609a0a80cb,"Edit Request \n\nHello, guys FUCK U pieces of ...",0.126582,edit request hello guys fuck u pieces of sh...,"['Edit', 'Request', 'Hello', 'guys', 'FUCK', '...",edit request hello guys fuck u pieces of shit ...,edit request hello guys fuck u pieces shit hat...,edit request hello guys fuck u pieces shit hat...,edit request hello guy fuck u piece shit hate ...,edit request hello guy fuck u piece shit hate ...,edit request hello guy fuck u piece of shit i ...,edit request hello guy fuck u piece of shit i ...,1,1,1,0,1,0
57,145430,1ca32e3575945845,ROBERT ULLMANN SUCKS DICK,0.880000,robert ullmann sucks dick,"['ROBERT', 'ULLMANN', 'SUCKS', 'DICK']",robert ullmann sucks dick,robert ullmann sucks dick,robert ullmann sucks dick,robert ullmann suck dick,robert ullmann suck dick,robert ullmann suck dick,robert ullmann suck dick,1,0,1,0,1,0
71,104720,30358dda4c531390,You have a problem with me or something? Jeez...,0.030075,you have a problem with me or something jeez...,"['You', 'have', 'a', 'problem', 'with', 'me', ...",you have a problem with me or something jeeze ...,problem something jeeze never anything harassi...,problem something jeeze never anything harassi...,problem something jeeze never anything harass ...,problem something jeeze never anything harass ...,you have a problem with me or something jeeze ...,you have a problem with me or something jeeze ...,1,0,0,0,0,0


In [69]:
data1.shape

(20000, 19)

In [80]:
vectorizer = CountVectorizer(ngram_range=(1,1),max_features=1000)

X_toxic, toxic_terms = vectorizer.fit_transform(data1['comment_cleaned_spell_no_stopwords_lemm']), vectorizer.get_feature_names_out()
toxic_countvectorizer = pd.DataFrame(X_toxic.toarray(), columns=toxic_terms)
print(f"toxic countvectorizer: {toxic_countvectorizer.shape}")
toxic_countvectorizer.head(5)

toxic countvectorizer: (20000, 1000)


,_child_,_datetime_,_emoji_,_ipaddress_,_number_,_tag_,_url_,able,absolutely,abuse,...,ya,yaaa,yaaaa,yeah,year,yes,yet,yo,youbollocks,yourselfgo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [81]:
# toxic_terms

In [82]:
#logistic regression and (1,1)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
vectorizer = CountVectorizer(ngram_range=(1,1), max_features=1000)
X = vectorizer.fit_transform(data1['comment_cleaned_spell_no_stopwords_lemm'])
y = data1["toxic"]
print(f"Shape of dataframe is {X.shape}")
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
logistic_regression1 = LogisticRegression(max_iter=1000)
logistic_regression1.fit(X_train, y_train)
training_predictions = logistic_regression1.predict(X_train)
testing_predictions = logistic_regression1.predict(X_test)
print('recall',recall_score(y_test, testing_predictions))
print('precision',precision_score(y_test, testing_predictions))
print('confusion matrix',confusion_matrix(y_test, testing_predictions))
print('roc_auc score',roc_auc_score(y_test, logistic_regression1.predict_proba(X_test)[:, 1]))

Shape of dataframe is (20000, 1000)
recall 0.8284214775938635
precision 0.8996054362121877
confusion matrix [[2294  229]
 [ 425 2052]]
roc_auc score 0.9354621375153191


In [83]:
#logistic regression and (2,2)
from sklearn.linear_model import LogisticRegression
vectorizer = CountVectorizer(ngram_range=(2,2), max_features=1000)
X = vectorizer.fit_transform(data1['comment_cleaned_spell_no_stopwords_lemm'])
y = data1["toxic"]
print(f"Shape of dataframe is {X.shape}")
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
logistic_regression2 = LogisticRegression(max_iter=1000)
logistic_regression2.fit(X_train, y_train)
training_predictions = logistic_regression2.predict(X_train)
testing_predictions = logistic_regression2.predict(X_test)
print('recall',recall_score(y_test, testing_predictions))
print('precision',precision_score(y_test, testing_predictions))
print('confusion matrix',confusion_matrix(y_test, testing_predictions))
print('roc_auc score',roc_auc_score(y_test, logistic_regression2.predict_proba(X_test)[:, 1]))

Shape of dataframe is (20000, 1000)
recall 0.8534517561566412
precision 0.602966343411295
confusion matrix [[1131 1392]
 [ 363 2114]]
roc_auc score 0.7230870420872423


In [84]:
#logistic regression and (3,3)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
# data1['lemmatized_review'] = data1['comment_text'].apply(lemmatize)
vectorizer = CountVectorizer(ngram_range=(3,3), max_features=1000)
X = vectorizer.fit_transform(data1['comment_cleaned_spell_no_stopwords_lemm'])
y = data1["toxic"]
print(f"Shape of dataframe is {X.shape}")
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
logistic_regression3 = LogisticRegression(max_iter=1000)
logistic_regression3.fit(X_train, y_train)
training_predictions = logistic_regression3.predict(X_train)
testing_predictions = logistic_regression3.predict(X_test)
print('recall',recall_score(y_test, testing_predictions))
print('precision',precision_score(y_test, testing_predictions))
print('confusion matrix',confusion_matrix(y_test, testing_predictions))
print('roc_auc score',roc_auc_score(y_test, logistic_regression3.predict_proba(X_test)[:, 1]))

Shape of dataframe is (20000, 1000)
recall 0.968106580540977
precision 0.5225539333188058
confusion matrix [[ 332 2191]
 [  79 2398]]
roc_auc score 0.5624311241703498


In [85]:
#random forest and (1,1)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
# data1['lemmatized_review'] = data1['comment_text'].apply(lemmatize)
vectorizer = CountVectorizer(ngram_range=(1,1), max_features=1000)
X = vectorizer.fit_transform(data1['comment_cleaned_spell_no_stopwords_lemm'])
y = data1["toxic"]
print(f"Shape of dataframe is {X.shape}")
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 3)
rf_model1=RandomForestClassifier(n_estimators=250)
rf_model1.fit(X_train,y_train)
training_predictions = rf_model1.predict(X_train)
testing_predictions = rf_model1.predict(X_test)
print('recall',recall_score(y_test, testing_predictions))
print('precision',precision_score(y_test, testing_predictions))
print('confusion matrix',confusion_matrix(y_test, testing_predictions))
print('roc_auc score',roc_auc_score(y_test, rf_model1.predict_proba(X_test)[:, 1]))


Shape of dataframe is (20000, 1000)
recall 0.8386968613428685
precision 0.8941126641253706
confusion matrix [[2233  250]
 [ 406 2111]]
roc_auc score 0.934970353029124


In [86]:
#random forest and (2,2)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
# data1['lemmatized_review'] = data1['comment_text'].apply(lemmatize)
vectorizer = CountVectorizer(ngram_range=(2,2), max_features=1000)
X = vectorizer.fit_transform(data1['comment_cleaned_spell_no_stopwords_lemm'])
y = data1["toxic"]
print(f"Shape of dataframe is {X.shape}")
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 3)
rf_model2=RandomForestClassifier(n_estimators=250)
rf_model2.fit(X_train,y_train)
training_predictions = rf_model2.predict(X_train)
testing_predictions = rf_model2.predict(X_test)
print('recall',recall_score(y_test, testing_predictions))
print('precision',precision_score(y_test, testing_predictions))
print('confusion matrix',confusion_matrix(y_test, testing_predictions))
print('roc_auc score',roc_auc_score(y_test, rf_model2.predict_proba(X_test)[:, 1]))


Shape of dataframe is (20000, 1000)
recall 0.8438617401668653
precision 0.6210526315789474
confusion matrix [[1187 1296]
 [ 393 2124]]
roc_auc score 0.7290054692128964


In [87]:
#random forest and (3,3)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
# data1['lemmatized_review'] = data1['comment_text'].apply(lemmatize)
vectorizer = CountVectorizer(ngram_range=(3,3), max_features=1000)
X = vectorizer.fit_transform(data1['comment_cleaned_spell_no_stopwords_lemm'])
y = data1["toxic"]
print(f"Shape of dataframe is {X.shape}")
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 3)
rf_model3=RandomForestClassifier(n_estimators=250)
rf_model3.fit(X_train,y_train)
training_predictions = rf_model3.predict(X_train)
testing_predictions = rf_model3.predict(X_test)
print('recall',recall_score(y_test, testing_predictions))
print('precision',precision_score(y_test, testing_predictions))
print('confusion matrix',confusion_matrix(y_test, testing_predictions))
print('roc_auc score',roc_auc_score(y_test, rf_model3.predict_proba(X_test)[:, 1]))


Shape of dataframe is (20000, 1000)
recall 0.9694080254270957
precision 0.5334499344118933
confusion matrix [[ 349 2134]
 [  77 2440]]
roc_auc score 0.5687207776487585


# Full Dataset

In [40]:
#logistic regression and (1,1)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
vectorizer = CountVectorizer(ngram_range=(1,1), max_features=1000)
X = vectorizer.fit_transform(data['comment_cleaned_spell_no_stopwords_lemm'])
y = data["toxic"]
print(f"Shape of dataframe is {X.shape}")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state = 1)
logistic_regression1 = LogisticRegression(max_iter=1000)
logistic_regression1.fit(X_train, y_train)
training_predictions = logistic_regression1.predict(X_train)
testing_predictions = logistic_regression1.predict(X_test)
print('recall',recall_score(y_test, testing_predictions))
print('precision',precision_score(y_test, testing_predictions))
print('confusion matrix',confusion_matrix(y_test, testing_predictions))
print('roc_auc score',roc_auc_score(y_test, logistic_regression1.predict_proba(X_test)[:, 1]))

Shape of dataframe is (127637, 1000)
recall 0.5157207023274806
precision 0.8668496911461908
confusion matrix [[22885   194]
 [ 1186  1263]]
roc_auc score 0.9213871996926565


In [41]:
#logistic regression and (2,2)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
vectorizer = CountVectorizer(ngram_range=(2,2), max_features=1000)
X = vectorizer.fit_transform(data['comment_cleaned_spell_no_stopwords_lemm'])
y = data["toxic"]
print(f"Shape of dataframe is {X.shape}")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state = 1)
logistic_regression2 = LogisticRegression(max_iter=1000)
logistic_regression2.fit(X_train, y_train)
training_predictions = logistic_regression2.predict(X_train)
testing_predictions = logistic_regression2.predict(X_test)
print('recall',recall_score(y_test, testing_predictions))
print('precision',precision_score(y_test, testing_predictions))
print('confusion matrix',confusion_matrix(y_test, testing_predictions))
print('roc_auc score',roc_auc_score(y_test, logistic_regression2.predict_proba(X_test)[:, 1]))

Shape of dataframe is (127637, 1000)
recall 0.09922417313189057
precision 0.8408304498269896
confusion matrix [[23033    46]
 [ 2206   243]]
roc_auc score 0.7295938669725524


In [42]:
#logistic regression and (3,3)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
vectorizer = CountVectorizer(ngram_range=(3,3), max_features=1000)
X = vectorizer.fit_transform(data['comment_cleaned_spell_no_stopwords_lemm'])
y = data["toxic"]
print(f"Shape of dataframe is {X.shape}")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state = 1)
logistic_regression3 = LogisticRegression(max_iter=1000)
logistic_regression3.fit(X_train, y_train)
training_predictions = logistic_regression3.predict(X_train)
testing_predictions = logistic_regression3.predict(X_test)
print('recall',recall_score(y_test, testing_predictions))
print('precision',precision_score(y_test, testing_predictions))
print('confusion matrix',confusion_matrix(y_test, testing_predictions))
print('roc_auc score',roc_auc_score(y_test, logistic_regression3.predict_proba(X_test)[:, 1]))

Shape of dataframe is (127637, 1000)
recall 0.008574928542262147
precision 0.84
confusion matrix [[23075     4]
 [ 2428    21]]
roc_auc score 0.5682155231862805


In [43]:
#random forest and (1,1)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
vectorizer = CountVectorizer(ngram_range=(1,1), max_features=1000)
X = vectorizer.fit_transform(data['comment_cleaned_spell_no_stopwords_lemm'])
y = data["toxic"]
print(f"Shape of dataframe is {X.shape}")
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 3)
rf_model1=RandomForestClassifier(n_estimators=250)
rf_model1.fit(X_train,y_train)
training_predictions = rf_model1.predict(X_train)
testing_predictions = rf_model1.predict(X_test)
print('recall',recall_score(y_test, testing_predictions))
print('precision',precision_score(y_test, testing_predictions))
print('confusion matrix',confusion_matrix(y_test, testing_predictions))
print('roc_auc score',roc_auc_score(y_test, rf_model1.predict_proba(X_test)[:, 1]))


Shape of dataframe is (127637, 1000)
recall 0.5906548206646923
precision 0.8226397800183318
confusion matrix [[28484   387]
 [ 1244  1795]]
roc_auc score 0.9097526208679294


In [44]:
#random forest and (2,2)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
vectorizer = CountVectorizer(ngram_range=(2,2), max_features=1000)
X = vectorizer.fit_transform(data['comment_cleaned_spell_no_stopwords_lemm'])
y = data["toxic"]
print(f"Shape of dataframe is {X.shape}")
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 3)
rf_model2=RandomForestClassifier(n_estimators=250)
rf_model2.fit(X_train,y_train)
training_predictions = rf_model2.predict(X_train)
testing_predictions = rf_model2.predict(X_test)
print('recall',recall_score(y_test, testing_predictions))
print('precision',precision_score(y_test, testing_predictions))
print('confusion matrix',confusion_matrix(y_test, testing_predictions))
print('roc_auc score',roc_auc_score(y_test, rf_model2.predict_proba(X_test)[:, 1]))


Shape of dataframe is (127637, 1000)
recall 0.1263573543928924
precision 0.7164179104477612
confusion matrix [[28719   152]
 [ 2655   384]]
roc_auc score 0.711079674300709


In [45]:
#random forest and (3,3)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
vectorizer = CountVectorizer(ngram_range=(3,3), max_features=1000)
X = vectorizer.fit_transform(data['comment_cleaned_spell_no_stopwords_lemm'])
y = data["toxic"]
print(f"Shape of dataframe is {X.shape}")
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 3)
rf_model3=RandomForestClassifier(n_estimators=250)
rf_model3.fit(X_train,y_train)
training_predictions = rf_model3.predict(X_train)
testing_predictions = rf_model3.predict(X_test)
print('recall',recall_score(y_test, testing_predictions))
print('precision',precision_score(y_test, testing_predictions))
print('confusion matrix',confusion_matrix(y_test, testing_predictions))
print('roc_auc score',roc_auc_score(y_test, rf_model3.predict_proba(X_test)[:, 1]))

Shape of dataframe is (127637, 1000)
recall 0.015136558078315236
precision 0.8070175438596491
confusion matrix [[28860    11]
 [ 2993    46]]
roc_auc score 0.5680998143481717


#on test set

In [88]:
datatest = pd.read_csv('/content/drive/Shareddrives/NLP/Team Blue NLP/test_cleaned.csv')
datatest.head()

,Unnamed: 0,id,comment_text,avg_count_caps,comment_cleaned,comment_tokenized,comment_cleaned_spell,comment_cleaned_no_stopwords,comment_cleaned_spell_no_stopwords,comment_cleaned_no_stopwords_lemm,comment_cleaned_spell_no_stopwords_lemm,comment_cleaned_lemm,comment_cleaned_spell_lemm,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,99174,127af536a900ba93,"According to Article I, the Vice President of ...",0.065574,according to article i the vice president of ...,"['According', 'to', 'Article', 'I', 'the', 'Vi...",according to article i the vice president of t...,according article vice president us president ...,according article vice president us president ...,accord article vice president us president sen...,accord article vice president us president sen...,accord to article i the vice president of the ...,accord to article i the vice president of the ...,0,0,0,0,0,0
1,3121,086c163d9ba75c28,Names of the individual games \n\nPutting in n...,0.029070,names of the individual games putting in nam...,"['Names', 'of', 'the', 'individual', 'games', ...",names of the individual games putting in names...,names individual games putting names individua...,names individual games putting names individua...,name individual game put name individual game ...,name individual game put name individual game ...,name of the individual game put in name of ind...,name of the individual game put in name of ind...,0,0,0,0,0,0
2,73540,c4cdee12f0b181df,Does anybody know the exact birth date?,0.025641,does anybody know the exact birth date,"['Does', 'anybody', 'know', 'the', 'exact', 'b...",does anybody know the exact birth date,anybody know exact birth date,anybody know exact birth date,anybody know exact birth date,anybody know exact birth date,do anybody know the exact birth date,do anybody know the exact birth date,0,0,0,0,0,0
3,5694,0f3bdf9067d194e3,Typical examples would be documents (e.g. spec...,0.014981,typical examples would be documents e g spec...,"['Typical', 'examples', 'would', 'be', 'docume...",typical examples would be documents e g specif...,typical examples would documents e g specifica...,typical examples would documents e g specifica...,typical example would document e g specificati...,typical example would document e g specificati...,typical example would be document e g specific...,typical example would be document e g specific...,0,0,0,0,0,0
4,33814,5a2e7d3a7655628d,"- you are duing very good works, good informat...",0.000000,you are duing very good works good informat...,"['you', 'are', 'duing', 'very', 'good', 'works...",you are duing very good works good information...,duing good works good informations keep mastar...,duing good works good informations keep mastar...,duing good work good information keep mastar w...,duing good work good information keep mastar w...,you be duing very good work good information k...,you be duing very good work good information k...,0,0,0,0,0,0


In [89]:
datatest.shape

(31915, 19)

In [90]:
datatest.dropna(inplace=True)

In [91]:
#logistic regression (1,1) with test
vectorizer = CountVectorizer(ngram_range=(1,1), max_features=1000)
X = vectorizer.fit_transform(datatest['comment_cleaned_spell_no_stopwords_lemm'])
y = datatest["toxic"]
testing_predictions = logistic_regression1.predict(X)
print('recall',recall_score(y, testing_predictions))
print('precision',precision_score(y, testing_predictions))
print('confusion matrix',confusion_matrix(y, testing_predictions))
print('roc_auc score',roc_auc_score(y, logistic_regression1.predict_proba(X)[:, 1]))
print('F1 score', f1_score(y, testing_predictions))

recall 0.3533834586466165
precision 0.08616292045273394
confusion matrix [[17387 11465]
 [ 1978  1081]]
roc_auc score 0.4832480283886831
F1 score 0.13854533803268185


In [92]:
#logistic regression (2,2) with test
vectorizer = CountVectorizer(ngram_range=(2,2), max_features=1000)
X = vectorizer.fit_transform(datatest['comment_cleaned_spell_no_stopwords_lemm'])
y = datatest["toxic"]
testing_predictions = logistic_regression2.predict(X)
print('recall',recall_score(y, testing_predictions))
print('precision',precision_score(y, testing_predictions))
print('confusion matrix',confusion_matrix(y, testing_predictions))
print('roc_auc score',roc_auc_score(y, logistic_regression2.predict_proba(X)[:, 1]))
print('F1 score', f1_score(y, testing_predictions))

recall 0.8192219679633868
precision 0.10343831262640855
confusion matrix [[ 7131 21721]
 [  553  2506]]
roc_auc score 0.4933466120137323
F1 score 0.1836839404822986


In [93]:
#logistic regression (3,3) with test
vectorizer = CountVectorizer(ngram_range=(3,3), max_features=1000)
X = vectorizer.fit_transform(datatest['comment_cleaned_spell_no_stopwords_lemm'])
y = datatest["toxic"]
testing_predictions = logistic_regression3.predict(X)
print('recall',recall_score(y, testing_predictions))
print('precision',precision_score(y, testing_predictions))
print('confusion matrix',confusion_matrix(y, testing_predictions))
print('roc_auc score',roc_auc_score(y, logistic_regression3.predict_proba(X)[:, 1]))
print('F1 score', f1_score(y, testing_predictions))

recall 0.9790781301078784
precision 0.10212432229685955
confusion matrix [[ 2520 26332]
 [   64  2995]]
roc_auc score 0.5345479303989967
F1 score 0.18495646266905455


In [94]:
#random forest (1,1) with test
vectorizer = CountVectorizer(ngram_range=(1,1), max_features=1000)
X = vectorizer.fit_transform(datatest['comment_cleaned_spell_no_stopwords_lemm'])
y = datatest["toxic"]
testing_predictions = rf_model1.predict(X)
print('recall',recall_score(y, testing_predictions))
print('precision',precision_score(y, testing_predictions))
print('confusion matrix',confusion_matrix(y, testing_predictions))
print('roc_auc score',roc_auc_score(y, rf_model1.predict_proba(X)[:, 1]))
print('F1 score', f1_score(y, testing_predictions))

recall 0.5658711997384767
precision 0.10427710843373494
confusion matrix [[13983 14869]
 [ 1328  1731]]
roc_auc score 0.5377972123812809
F1 score 0.1761025484510911


In [95]:
#random forest (2,2) with test
vectorizer = CountVectorizer(ngram_range=(2,2), max_features=1000)
X = vectorizer.fit_transform(datatest['comment_cleaned_spell_no_stopwords_lemm'])
y = datatest["toxic"]
testing_predictions = rf_model2.predict(X)
print('recall',recall_score(y, testing_predictions))
print('precision',precision_score(y, testing_predictions))
print('confusion matrix',confusion_matrix(y, testing_predictions))
print('roc_auc score',roc_auc_score(y, rf_model2.predict_proba(X)[:, 1]))
print('F1 score', f1_score(y, testing_predictions))

recall 0.8094148414514547
precision 0.11037803138373752
confusion matrix [[ 8896 19956]
 [  583  2476]]
roc_auc score 0.5440488986255656
F1 score 0.19426464242281588


In [62]:
#random forest (3,3) with test
vectorizer = CountVectorizer(ngram_range=(3,3), max_features=1000)
X = vectorizer.fit_transform(datatest['comment_cleaned_spell_no_stopwords_lemm'])
y = datatest["toxic"]
testing_predictions = rf_model3.predict(X)
print('recall',recall_score(y, testing_predictions))
print('precision',precision_score(y, testing_predictions))
print('confusion matrix',confusion_matrix(y, testing_predictions))
print('roc_auc score',roc_auc_score(y, rf_model3.predict_proba(X)[:, 1]))
print('F1 score', f1_score(y, testing_predictions))

recall 0.003922850604772802
precision 0.06521739130434782
confusion matrix [[28680   172]
 [ 3047    12]]
roc_auc score 0.5358703900692907
F1 score 0.007400555041628122
